***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-5660  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 29 06:09 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = '318305570_207757535_bucket'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd
filtered_doc_text_pairs = doc_text_pairs.filter(lambda x: "genetics" in x[0])

24/02/29 06:18:04 WARN TaskSetManager: Lost task 0.0 in stage 1.0 (TID 3485) (cluster-5660-w-2.c.irfinalproject-415108.internal executor 4): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:375)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:517)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:565)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:558)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasInParallel$2(SchemaMergeUtils.scala:76)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:855)
	at org.apache.spark.rdd.RDD.$anonfun$ma

Py4JJavaError: An error occurred while calling o81.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 4 times, most recent failure: Lost task 0.3 in stage 1.0 (TID 3488) (cluster-5660-w-0.c.irfinalproject-415108.internal executor 6): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:375)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:517)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1(ParquetFileFormat.scala:565)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.$anonfun$mergeSchemasInParallel$1$adapted(ParquetFileFormat.scala:558)
	at org.apache.spark.sql.execution.datasources.SchemaMergeUtils$.$anonfun$mergeSchemasIn

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_body_gcp_training.py

inverted_index_body_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_body_gcp_training.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_body_gcp_training import *

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes:
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab.

In [14]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in
  `all_stopwords` and return entries that will go into our posting lists.
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs
      for example: [("Anarchism", (12, 5)), ...]
  '''
  word_freq = {}
  for token in text:
      if token not in all_stopwords:
          if token in word_freq:
              word_freq[token] += 1
          else:
              word_freq[token] = 1
  return [(token, (id, freq)) for token, freq in word_freq.items()]

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # Sort the unsorted posting list by wiki_id
  sorted_pl = sorted(unsorted_pl, key=lambda x: x[0])
  return sorted_pl

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # Count the number of documents each token appears in
  token_df = postings.map(lambda x: (x[0], len(x[1]))).distinct()
  return token_df

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''
  # Partition postings into buckets
  bucketed_postings = postings.groupBy(lambda x: token2bucket_id(x[0]))

  # Write posting lists to disk and get posting locations for each bucket
  posting_locations = bucketed_postings.map(lambda x: InvertedIndex.write_a_posting_list((x[0], list(x[1])),'.', bucket_name))

  return posting_locations

In [15]:
tokens_doc_pairs = filtered_doc_text_pairs.map(lambda x: ([token.group() for token in RE_WORD.finditer(x[0].lower())], x[1]))
word_counts = tokens_doc_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>10)
filtered_rdd = postings_filtered.flatMap(lambda x: [(item[0], (x[0], item[1])) for item in x[1]]) \
                      .groupByKey() \
                      .map(lambda x: (x[0], list(x[1])))
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(postings_filtered).collect()

In [16]:
# test index construction time
print(postings_filtered.take(2))
print(filtered_rdd.take(2))

[('common', [(25, 11), (569, 1), (627, 3), (698, 2), (777, 2), (786, 6), (852, 1), (874, 9), (1910, 2), (1911, 2), (1914, 14), (1916, 2), (1997, 1), (2154, 14), (2472, 1), (2494, 3), (2594, 4), (2786, 1), (2864, 5), (2909, 2), (2965, 20), (3029, 1), (3034, 7), (3209, 1), (3416, 7), (3717, 6), (3878, 4), (3954, 8), (3974, 5), (3978, 8), (4183, 6), (4185, 2), (4211, 2), (4213, 6), (4214, 5), (4292, 3), (4402, 9), (4543, 9), (4583, 3), (4620, 4), (4805, 1), (4868, 1), (5009, 3), (5131, 2), (5170, 1), (5180, 4), (5221, 1), (5321, 2), (5326, 9), (5331, 3), (5376, 17), (5717, 4), (5823, 2), (5876, 10), (6080, 2), (6335, 1), (6339, 2), (6438, 3), (6546, 12), (6621, 2), (6682, 9), (6710, 11), (6736, 1), (6828, 3), (6868, 3), (6910, 4), (6949, 4), (7172, 14), (7316, 8), (7329, 10), (7626, 8), (7638, 1), (7844, 8), (7955, 6), (8013, 2), (8129, 4), (8303, 25), (8305, 4), (8311, 26), (8389, 15), (8411, 1), (8449, 1), (8498, 5), (8703, 1), (9015, 1), (9032, 1), (9236, 39), (9238, 1), (9334, 2), (93

[(7316, [('common', 8), ('origin', 2), ('larger', 1), ('periods', 4), ('remarks', 1), ('identified', 2), ('complexity', 2), ('severely', 1), ('suited', 3), ('kinds', 1), ('atmosphere', 7), ('hydrocarbon', 4), ('2011', 2), ('alternative', 26), ('rapidly', 1), ('limiting', 1), ('inhabit', 1), ('silicon', 39), ('ribosomes', 1), ('observed', 4), ('necessary', 3), ('electron', 3), ('north', 1), ('ones', 2), ('lake', 1), ('exist', 14), ('1915', 2), ('dissolve', 5), ('biosphere', 13), ('substantial', 1), ('result', 1), ('catalyst', 2), ('raised', 2), ('zone', 1), ('joined', 1), ('supports', 1), ('dissolving', 1), ('vapour', 1), ('fluid', 1), ('iron', 3), ('mixture', 3), ('arsenic', 9), ('conceivable', 2), ('grow', 1), ('engineered', 1), ('2005', 1), ('enough', 2), ('supporting', 1), ('complex', 7), ('increasing', 1), ('roughly', 3), ('donor', 1), ('devised', 2), ('easier', 1), ('amide', 1), ('α-helices', 1), ('suggested', 7), ('purely', 2), ('possibility', 4), ('sufficiently', 2), ('stages', 

In [17]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [22]:
import math
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
#partition_func
inverted.posting_locs = super_posting_locs
inverted.doc_count = filtered_rdd.count()
N = inverted.doc_count
# inverted._posting_list = dict(postings_filtered.collect())
##############################################################
##### need to change filtered_doc_text_pairs to doc_text_pairs
##### when testing the whole corpus
##############################################################
# inverted.doc_count = filtered_doc_text_pairs.count()
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# Iterate through tokens_doc_pairs to compute TF-IDF for each token in each document
sum_len_of_all_docs = 0
for doc_id, words_in_doc in filtered_rdd.take(N):
  sum_len_docs = 0
  sum_doc_tf_idf = 0
  for token, tf in words_in_doc:
    if inverted.df.get(token) is not None:
      df = inverted.df[token]
      idf = math.log(inverted.doc_count / df, 10)
      sum_len_docs += tf
      sum_doc_tf_idf += math.pow(tf*idf, 2)
  inverted.doc_tf_idf_doc_len[doc_id] = (math.sqrt(sum_doc_tf_idf), sum_len_docs)
  sum_len_of_all_docs += sum_len_docs
inverted.doc_len_avg = sum_len_of_all_docs / inverted.doc_count
inverted.write_index('.', 'index_body_genetics')
# upload to gs
index_src = "index_body_genetics.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index_body_genetics.pkl [Content-Type=application/octet-stream]...
/ [1 files][815.7 KiB/815.7 KiB]                                                
Operation completed over 1 objects/815.7 KiB.                                    


In [24]:
temp_index = InvertedIndex.read_index('.', 'index_body_genetics',bucket_name)
print(temp_index.df.items())

dict_items([('tarrant', 11), ('negros', 19), ('mesa', 31), ('beta-thalassemia', 11), ('modalities', 76), ('insectivore', 11), ('prescribe', 51), ("navy's", 15), ('minimization', 27), ('larger', 1998), ('phylogenetics', 1802), ('frankie', 14), ('ceos', 22), ('477', 38), ('etruscan', 24), ('deflect', 16), ('labour-intensive', 13), ('secured', 161), ('postural', 25), ('eosinophilic', 28), ('lipman', 16), ('bayer', 54), ('crawford', 98), ('bourne', 30), ('bona', 18), ('penicillin', 87), ('schmitt', 31), ('possibly', 1369), ('calculating', 133), ('glider', 29), ('controversial', 824), ('amnesiac', 24), ('backbone', 184), ('biome', 32), ('demethylases', 23), ('ascertained', 38), ('unrealistic', 46), ('general-purpose', 18), ('modernism', 27), ('reconnected', 15), ('incandescent', 23), ('malus', 11), ('tone', 244), ('whyte', 16), ('hinges', 20), ('mayoral', 12), ('launceston', 20), ('credits', 142), ('deformed', 74), ('eosinophil', 14), ('cromwell', 34), ('invasiveness', 18), ('nucleation', 2

In [23]:
!gsutil ls -lh $index_dst

815.74 KiB  2024-02-28T19:39:56Z  gs://318305570_207757535_bucket/postings_gcp/index_body_genetics.pkl
TOTAL: 1 objects, 835316 bytes (815.74 KiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
def generate_graph(pages):
    # Extract edges from pages
    edges = pages.flatMap(extract_edges).distinct()

    # Extract vertices from edges
    vertices = edges.flatMap(lambda edge: ((edge[0],), (edge[1],))).distinct()

    return edges, vertices

def extract_edges(row):
    edges = []
    for anchor_text in row[1]:
          edges.append((row[0], anchor_text[0]))
    return edges

In [ ]:
t_start = time()
pages_links = parquetFile.select ("id","anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

24/02/16 17:30:02 WARN YarnAllocator: Container from a bad node: container_1708100150783_0001_01_000012 on host: cluster-d9c8-w-1.c.bubbly-bastion-414507.internal. Exit status: 143. Diagnostics: [2024-02-16 17:30:01.385]Container killed on request. Exit code is 143
[2024-02-16 17:30:01.386]Container exited with a non-zero exit code 143. 
[2024-02-16 17:30:01.386]Killed by external signal
.
24/02/16 17:30:02 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 12 for reason Container from a bad node: container_1708100150783_0001_01_000012 on host: cluster-d9c8-w-1.c.bubbly-bastion-414507.internal. Exit status: 143. Diagnostics: [2024-02-16 17:30:01.385]Container killed on request. Exit code is 143
[2024-02-16 17:30:01.386]Container exited with a non-zero exit code 143. 
[2024-02-16 17:30:01.386]Killed by external signal
.
24/02/16 17:30:02 ERROR YarnScheduler: Lost executor 12 on cluster-d9c8-w-1.c.bubbly-bastion-414507.internal: Container from a bad nod

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750|  9913.72878216077|
|  10568| 5385.349263642037|
|  32927| 5282.081575765277|
|  30680| 5128.233709604119|
|5843419| 4957.567686263868|
|  68253| 4769.278265355163|
|  31717|  4486.35018054831|
|  11867|4146.4146509127695|
|  14533| 3996.466440885503|
| 645042|3531.6270898037437|
|  17867|3246.0983906041406|
|5042916| 2991.945739166176|
|4689264|2982.3248830417474|
|  14532|2934.7468292031704|
|  25391|2903.5462235133978|
|   5405|2891.4163291546356|
|4764461| 2834.366987332661|
|  15573|  2783.86511815884|
|   9316|  2782.03964641377|
|8569916|2775.2861918400167|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*120

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete
the reporting requirements for assignment #3.

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://318305570_netta/postings_gcp


In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0.011
print(f'I used {cost} USD credit during the course of this assignment')

I used 0.011 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour.


In [ ]:
#If you have decided to do the bonus task - please copy the code here

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag